In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import fix_yahoo_finance as yf
import datetime
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

In [2]:
# end day is today
today = datetime.datetime.today()
today.strftime('%m/%d/%Y')

# start day is 5 business days ago
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
lastBusDay = today - 6 * US_BUSINESS_DAY
lastBusDay.strftime('%m/%d/%Y')

'08/31/2018'

## Calculating daily price movements for all stocks

In [3]:
SP = web.DataReader('^GSPC', 'yahoo', lastBusDay, today)
SP = SP.iloc[::-1]
# generate dollar value of movement between adjacent business days
SP['Mov $'] = round(-SP['Close'].diff().shift(-1), 2)
# generate percentage of movement between adjacent business days
SP['Mov %'] = SP['Mov $']/SP['Close'].shift(-1)
SP = SP[:5].reset_index()

ticker_list = ["GOOG", "AMZN", "FB", "MSFT", "ADBE", "BABA", "BIDU", "BKNG", "BZUN", "MA", "MTCH", "MOMO", "NFLX", "PCTY", 
               "PYPL", "RP", "CRM", "SINA", "SQ", "ZG", "CTRP", "NVDA", "RNG", "WB", "ENV", "WUBA", "GDDY"]
df_final = pd.DataFrame()

for ticker in ticker_list:
    df = web.DataReader(ticker, 'yahoo', lastBusDay, today)
    df['Ticker'] = ticker
    df = df.iloc[::-1]
    # generate dollar value of movement between adjacent business days
    df['Mov $'] = round(-df['Close'].diff().shift(-1), 2)
    # generate percentage of movement between adjacent business days
    df['Mov %'] = df['Mov $']/df['Close'].shift(-1)
    # movement percentage get rid of market fluctuation
    # --> df['Mov_noSP %'] = pd.Series(["{0:.2f}%".format(X) for X in (df['Mov %'] - SP['Mov %'])]) 
    df_final = df_final.append(df[:5])
#len(df_final)
df_final.reset_index()

SP_final = SP[['Date','Mov %']]
df_final = df_final.merge(SP_final, on='Date', how='left').rename(index=str, columns={
    "Mov %_x": "Mov %_ST", "Mov %_y": "Mov %_SP"})
df_final['Mov %']=df_final['Mov %_ST']-df_final['Mov %_SP']
df_final['Mov %'] = pd.Series(["{0:.2f}%".format(X * 100) for X in df_final['Mov %']], index = df_final.index)
df_final['Mov %_ST'] = pd.Series(["{0:.2f}%".format(X * 100) for X in df_final['Mov %_ST']], index = df_final.index)
df_final['Mov %_SP'] = pd.Series(["{0:.2f}%".format(X * 100) for X in df_final['Mov %_SP']], index = df_final.index)
df_final

,Date,High,Low,Open,Close,Volume,Adj Close,Ticker,Mov $,Mov %_ST,Mov %_SP,Mov %
0,2018-09-11,1178.680054,1156.449951,1161.630005,1177.359985,1193670,1177.359985,GOOG,12.72,1.09%,0.37%,0.72%
1,2018-09-10,1174.540039,1160.109985,1172.189941,1164.640015,1115400,1164.640015,GOOG,-0.19,-0.02%,0.19%,-0.21%
2,2018-09-07,1175.260010,1157.214966,1158.670044,1164.829956,1401300,1164.829956,GOOG,-6.61,-0.56%,-0.22%,-0.34%
3,2018-09-06,1186.300049,1152.000000,1186.300049,1171.439941,1888500,1171.439941,GOOG,-15.04,-1.27%,-0.37%,-0.90%
4,2018-09-05,1199.010010,1162.000000,1193.800049,1186.479980,2061300,1186.479980,GOOG,-10.52,-0.88%,-0.28%,-0.60%
5,2018-09-11,1988.875000,1917.000000,1928.270020,1987.150024,4945640,1987.150024,AMZN,48.14,2.48%,0.37%,2.11%
6,2018-09-10,1973.040039,1931.520020,1971.000000,1939.010010,4513700,1939.010010,AMZN,-13.06,-0.67%,0.19%,-0.86%
7,2018-09-07,1975.199951,1937.349976,1938.709961,1952.069946,4892600,1952.069946,AMZN,-6.24,-0.32%,-0.22%,-0.10%
8,2018-09-06,2007.500000,1935.209961,2006.510010,1958.310059,7488700,1958.310059,AMZN,-36.51,-1.83%,-0.37%,-1.47%
9,2018-09-05,2040.380005,1989.890015,2038.109985,1994.819946,8220600,1994.819946,AMZN,-44.69,-2.19%,-0.28%,-1.91%


## Merge to Articles Data Frame

In [4]:
articles = pd.read_csv("AllArticlesOutput2018-09-11-12-31.csv", encoding = "ISO-8859-1")
articles['DateForm'] = pd.to_datetime(articles['Date']).dt.normalize()
article_w_price_change = (articles.merge(df_final, how='left', left_on=['Ticker','DateForm'], right_on=['Ticker','Date'])).drop(
    ['DateForm','High','Low','Open','Adj Close','Volume','Mov $','Mov %_ST','Mov %_SP'], axis=1)
article_w_price_change

,Ticker,Date_x,Title,Link,Text,Date_y,Close,Mov %
0,ADBE,09/10/18,Adobe to Webcast Q3 FY2018 Earnings Conference...,http://www.wsj.com/articles/PR-CO-20180910-907992,NaN,2018-09-10,265.369995,1.54%
1,ADBE,09/07/18,Watsons New Racket at the U.S. Open,https://www.barrons.com/articles/watsons-new-r...,NaN,2018-09-07,260.869995,0.65%
2,ADBE,09/07/18,The Nasdaqs Plunge Was a Selloff Waiting to H...,https://www.barrons.com/articles/the-nasdaqs-p...,Stocks slumped last weekand for once we cant...,2018-09-07,260.869995,0.65%
3,BABA,09/11/2018 11:53,Heed the early-warning signal in 11 popular te...,http://www.marketwatch.com/story/heed-the-earl...,Technology stocks have been the generals of th...,2018-09-11,157.460007,0.33%
4,BABA,09/11/2018 07:17,"Thinking about buying stock in American Tower,...",http://www.wsj.com/articles/PR-CO-20180911-906553,NaN,2018-09-11,157.460007,0.33%
5,BABA,09/11/2018 07:17,Alibaba Teams Up With Russian Tech Giant,https://www.wsj.com/articles/alibaba-teams-up-...,NaN,2018-09-11,157.460007,0.33%
6,BABA,09/10/2018 20:17,Alibabas Next Chairman Fancies Himself a Free...,https://www.wsj.com/articles/daniel-zhang-alib...,NaN,2018-09-10,156.360001,-3.89%
7,BABA,09/10/2018 14:17,Peak China?,https://www.wsj.com/articles/peak-china-153661...,NaN,2018-09-10,156.360001,-3.89%
8,BABA,09/10/18,Jack Ma Is Stepping Down at Alibaba. He Was a ...,https://www.barrons.com/articles/jack-ma-leavi...,HONG KONG -- As Chinas most famous celebrity ...,2018-09-10,156.360001,-3.89%
9,BABA,09/10/18,Jack Ma: Showman Who Has Shown China a New Way...,https://www.wsj.com/articles/jack-ma-a-showman...,NaN,2018-09-10,156.360001,-3.89%
